In [1]:
# Elevator Controller - Full Version (KRIA + PMOD)
# 功能：4樓層控制、Emergency、ADA速度控制、門紅綠燈顯示

from pynq import Overlay, GPIO
import time

# === Load Bitstream ===
overlay = Overlay("ps_gpio_kv260.bit")  # 替換成實際bitstream檔案

# === Button Inputs ===
floor_buttons = [
    GPIO(GPIO.get_gpio_pin(0), "in"),  # Floor 0
    GPIO(GPIO.get_gpio_pin(1), "in"),  # Floor 1
    GPIO(GPIO.get_gpio_pin(2), "in"),  # Floor 2
    GPIO(GPIO.get_gpio_pin(3), "in")   # Floor 3
]
emergency_button = GPIO(GPIO.get_gpio_pin(8), "in")

# === LED Outputs ===
floor_leds = [
    GPIO(GPIO.get_gpio_pin(4), "out"),  # Floor 0
    GPIO(GPIO.get_gpio_pin(5), "out"),  # Floor 1
    GPIO(GPIO.get_gpio_pin(6), "out"),  # Floor 2
    GPIO(GPIO.get_gpio_pin(7), "out")   # Floor 3
]
door_red = GPIO(GPIO.get_gpio_pin(9), "out")     # Door Closed LED
door_green = GPIO(GPIO.get_gpio_pin(11), "out")   # Door Open LED

# === State Definitions ===
IDLE = "IDLE"
MOVING = "MOVING"
DOOR_OPEN = "DOOR_OPEN"
EMERGENCY = "EMERGENCY"

# === Variables ===
state = IDLE
current_floor = 0
requests = [0, 0, 0, 0]

def update_leds():
    for i in range(4):
        floor_leds[i].write(1 if i == current_floor else 0)

def check_requests():
    for i, button in enumerate(floor_buttons):
        if button.read() == 1:
            requests[i] = 1

# === Main FSM Loop ===
while True:
    check_requests()

    if emergency_button.read() == 0:  # 反向邏輯：按下時為 0
        print("[EMERGENCY] Override all requests. Returning to Floor 0")
        requests = [0, 0, 0, 0]
        current_floor = 0
        state = EMERGENCY

    if state == EMERGENCY:
        update_leds()
        time.sleep(1)
        state = DOOR_OPEN

    elif state == IDLE:
        if any(requests):
            state = MOVING
        else:
            door_red.write(1)   # 門關紅燈 ON
            door_green.write(0) # 門開綠燈 OFF

    elif state == MOVING:
        for i in range(4):
            if requests[i] == 1:
                print(f"[MOVING] Going to floor {i}")
                requests[i] = 0
                steps = abs(current_floor - i)

                # === ADA 模擬速度控制 ===
                if steps == 0:
                    time.sleep(0.2)
                elif steps == 1:
                    time.sleep(0.3)
                    time.sleep(0.3)
                    time.sleep(0.3)
                elif steps == 2:
                    time.sleep(0.3)
                    time.sleep(0.6)
                    time.sleep(0.3)
                else:
                    time.sleep(0.3)
                    time.sleep(0.9)
                    time.sleep(0.3)

                current_floor = i
                state = DOOR_OPEN
                break

    elif state == DOOR_OPEN:
        print(f"[DOOR OPEN] Floor {current_floor}")
        update_leds()
        door_green.write(1)
        door_red.write(0)
        time.sleep(2)
        door_green.write(0)
        door_red.write(1)
        state = IDLE

    update_leds()
    time.sleep(0.1)

[EMERGENCY] Override all requests. Returning to Floor 0
[EMERGENCY] Override all requests. Returning to Floor 0
[EMERGENCY] Override all requests. Returning to Floor 0
[EMERGENCY] Override all requests. Returning to Floor 0
[EMERGENCY] Override all requests. Returning to Floor 0
[EMERGENCY] Override all requests. Returning to Floor 0
[EMERGENCY] Override all requests. Returning to Floor 0
[EMERGENCY] Override all requests. Returning to Floor 0
[EMERGENCY] Override all requests. Returning to Floor 0
[EMERGENCY] Override all requests. Returning to Floor 0
[EMERGENCY] Override all requests. Returning to Floor 0
[EMERGENCY] Override all requests. Returning to Floor 0
[EMERGENCY] Override all requests. Returning to Floor 0
[EMERGENCY] Override all requests. Returning to Floor 0
[EMERGENCY] Override all requests. Returning to Floor 0
[EMERGENCY] Override all requests. Returning to Floor 0
[EMERGENCY] Override all requests. Returning to Floor 0
[EMERGENCY] Override all requests. Returning to 

KeyboardInterrupt: 

未按： 0


按下： 0


In [5]:
from pynq import Overlay, GPIO
from time import sleep

overlay = Overlay("./ps_gpio_kv260.bit")

# GPIO
dir_led    = GPIO(GPIO.get_gpio_pin(0), 'out')
move_led   = GPIO(GPIO.get_gpio_pin(1), 'out')
door_led   = GPIO(GPIO.get_gpio_pin(2), 'out')
floor_led1 = GPIO(GPIO.get_gpio_pin(3), 'out')
floor_led2 = GPIO(GPIO.get_gpio_pin(4), 'out')
bit0 = GPIO(GPIO.get_gpio_pin(5), 'in')
bit1 = GPIO(GPIO.get_gpio_pin(6), 'in')
bit2 = GPIO(GPIO.get_gpio_pin(7), 'in')

# 狀態
floor_pos = 0
direction = 1
requests  = [0,0,0,0]
EMERGENCY = False
last_req  = [0,0,0,0]

# ---------- 顯示樓層 ----------
def show_floor(pos):
    if pos<100:   floor_led1.write(0); floor_led2.write(0); return 0
    if pos<200:   floor_led1.write(1); floor_led2.write(0); return 1
    if pos<300:   floor_led1.write(0); floor_led2.write(1); return 2
    floor_led1.write(1); floor_led2.write(1); return 3

# ---------- 按鈕 ----------
def read_buttons():
    global EMERGENCY, last_req
    code = f"{bit2.read()}{bit1.read()}{bit0.read()}"
    if   code=='001': requests[0]=1
    elif code=='010': requests[1]=1
    elif code=='011': requests[2]=1
    elif code=='100': requests[3]=1
    elif code=='101':
        EMERGENCY=True
        for i in range(1,4): requests[i]=0
        requests[0]=1
        print("‼ 緊急呼叫 (清空其他請求)")

    if requests != last_req:
        print("🆕 請求更新:", requests)
        last_req = requests.copy()

# ---------- 門 ----------
def door_cycle():
    door_led.write(1); print("🚪 開門")
    sleep(2)
    door_led.write(0); print("🚪 關門")

# ---------- 一單位移動 ----------
def move_unit(step, start_pos, target_pos):
    """
    step: +1 / -1
    start_pos: 起點絕對位置
    target_pos: 目標絕對位置
    return 到達樓層 idx 或 None
    """
    global floor_pos
    progress = abs(floor_pos - start_pos)        # 已走距離
    remain   = abs(target_pos - floor_pos)       # 剩餘距離
    is_down  = step == -1

        head_slow = 20 if is_down else 10  # 下行起步/結束慢速 20 單位，上行 10
    tail_slow = 20 if is_down else 10

    slow = (progress < head_slow) or (remain <= tail_slow)
    delay = 0.6 if slow else 0.1

    move_led.write(1); sleep(delay/2)
    move_led.write(0); sleep(delay/2)

    floor_pos += step
    floor_idx = floor_pos//100 if floor_pos%100==0 else None
    show_floor(floor_pos)
    return floor_idx

# ---------- 旅行 ----------
def travel_to(dest):
    global floor_pos
    step = 1 if dest > floor_pos//100 else -1
    start_pos  = floor_pos
    target_pos = dest*100
    print(f"➡ 前往 {dest+1}F")
    while floor_pos != target_pos:
        read_buttons()
        if EMERGENCY: print("↩ 被緊急打斷"); return False
        fl = move_unit(step, start_pos, target_pos)
        if fl is not None and requests[fl]:
            print(f"★ 停靠 {fl+1}F")
            requests[fl]=0
            door_cycle()
            return True
    print(f"★ 到達 {dest+1}F")
    door_cycle()
    return True

# ---------- FSM ----------
def fsm_once():
    global direction, EMERGENCY
    read_buttons()
    cur = floor_pos//100

    if EMERGENCY:                       # 緊急最高優先
        direction=0; dir_led.write(0)
        travel_to(0); EMERGENCY=False; return

    if direction:                       # 上行
        for f in range(cur,4):
            if requests[f]:
                dir_led.write(1)
                travel_to(f); return
        direction=0
    else:                               # 下行
        for f in range(cur-1,-1,-1):
            if requests[f]:
                dir_led.write(0)
                travel_to(f); return
        direction=1

print("🚀 Elevator controller (balanced slow start & stop) START")
while True:
    fsm_once()
    sleep(0.05)


IndentationError: unexpected indent (<ipython-input-5-0227cfdd4061>, line 67)

In [11]:
# 按鈕
floor_buttons = [
    GPIO(GPIO.get_gpio_pin(1), "in"),  # Floor‑1 按鈕
    GPIO(GPIO.get_gpio_pin(2), "in"),  # Floor‑2
    GPIO(GPIO.get_gpio_pin(3), "in")   # Floor‑3
]
emergency_button = GPIO(GPIO.get_gpio_pin(5), "in")   # Emergency (最下)

# LED
floor_leds = [
    GPIO(GPIO.get_gpio_pin(0), "out"),  # Floor‑0 LED (最下)
    GPIO(GPIO.get_gpio_pin(4), "out"),  # Floor‑1
    GPIO(GPIO.get_gpio_pin(6), "out"),  # Floor‑2
    GPIO(GPIO.get_gpio_pin(7), "out")   # Floor‑3
]
door_red   = GPIO(GPIO.get_gpio_pin(8),  "out")
door_green = GPIO(GPIO.get_gpio_pin(9),  "out")


In [12]:
# 按鈕
floor_buttons = [
    GPIO(GPIO.get_gpio_pin(1), "in"),  # Floor‑1 按鈕
    GPIO(GPIO.get_gpio_pin(2), "in"),  # Floor‑2
    GPIO(GPIO.get_gpio_pin(3), "in")   # Floor‑3
]
emergency_button = GPIO(GPIO.get_gpio_pin(5), "in")   # Emergency (最下)

# LED
floor_leds = [
    GPIO(GPIO.get_gpio_pin(0), "out"),  # Floor‑0 LED (最下)
    GPIO(GPIO.get_gpio_pin(4), "out"),  # Floor‑1
    GPIO(GPIO.get_gpio_pin(6), "out"),  # Floor‑2
    GPIO(GPIO.get_gpio_pin(7), "out")   # Floor‑3
]
door_red   = GPIO(GPIO.get_gpio_pin(8),  "out")
door_green = GPIO(GPIO.get_gpio_pin(9),  "out")
